Author: Irsyad Adam

Instructions on how to run notebook:
-   Make sure that entitycount.txt, caseolap.csv, and drug-categories.csv are all uploaded in Neo4j import filec
-   Make sure that in 'add_pathways_to_caseolap()' and 'edit_entity_count()', you specify locations of caseolap.csv and entitycount.txt on this directory
-   In the __init___ of graph_creation(), specify the bolt url and the unsername and password
-   Run the notebook!

In [2]:
from neo4j import GraphDatabase
import pandas as pd
from tqdm import tqdm
import requests
import pandas as pd

def extract_pathway(unid) -> str:
    """
    grabs all reactome pathways associated with protein
    @param id is the uniprot id that is going to be grabbed
    @return is a list with all of the reactome pathways
    """
    #get the url
    url = 'https://www.uniprot.org/uniprot/' + unid + '.txt'

    #check the response
    response = requests.get(url=url)

    #if successful
    if response.status_code == 200:
        pathway_list = []
        #new line delimiter
        response = response.text.splitlines()
        #searcg for pathways
        
        for i in range(len(response)):
            #if found
            if 'Reactome;' in response[i]:
                #process string
                pathway = response[i]
                pathway = str(pathway.split()[2][:-1])
                pathway_list.append(pathway)
        return pathway_list

    #if not successful
    else:
        #get error
        print('Error, Status Code:' % response.status_code)


def add_pathways(df) -> any:
    """
    appends reactome pathways to df
    @param df is dataframe with uniprot id
    @return df is df with newly added reactome column
    """
    pathway = []
    for i in tqdm(df['protein'], desc = "Searching Pathways in Caseolap CSV (__init__): "):
        result = extract_pathway(i)
        pathway.append(result)
    df['pathway'] = pathway
    return df


def add_pathways_to_caseolap(io = '../data/caseolap.csv') -> pd.DataFrame:
    '''
    adds pathways to caseolap.csv
    @param io is the filepath
    @return is a dataframe
    '''
    #grab pathways (80s)
    caseolap_df = pd.read_csv('../data/caseolap.csv')
    caseolap_df = add_pathways(caseolap_df) 
    return caseolap_df

def edit_entity_count(text = '../data/entitycount.txt') -> pd.DataFrame:
    '''
    turns entitycount.txt to a pandas df
    @param text is the entitycount
    @return is a df that returns the pmid and unid columns
    '''
    #read data
    data = open(text, 'r')
    data = [line.strip('\n') for line in data.readlines()]
    for i in range(len(data)):
        data[i] = data[i].split()

    #get pmids and unid
    pmid = [element[0] for element in data]
    unid = [[id for id in element[1:]] for element in data]

    #process unids
    for i in tqdm(range(len(unid)), desc = "Parsing PMID CSV (__init__): "):
        for j in range(len(unid[i])):
            #find the '|' character
            index = unid[i][j].find('|')

            #slice everything after that index
            unid[i][j] = unid[i][j][0:index]

    #load in everything into a dataframe
    df = {'pmid' : pmid, 'unid' : unid}
    df = pd.DataFrame(df)
    return df


#Have caseolap.csv, drug_category_target.csv, entitycount.csv in the imports folder in neo4j
class graph_creation:
    def __init__(self) -> None:
        self.driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "Ima62186Ima@"))
        self.pmid_df = edit_entity_count()
        self.df = []
        self.caseolap_df = add_pathways_to_caseolap()
        

    def close(self) -> None:
        self.driver.close()

    ###################################################################################################
    #constraints
    @classmethod
    def create_drug_contraints(cls, tx) -> None:
        """
        drug constraint query
        @param cls is the class
        @param tx is the transaction
        @returns none
        """
        query = ("CREATE CONSTRAINT ON (d:Drug) ASSERT d.name IS UNIQUE")
        tx.run(query)


    @classmethod
    def create_category_contraints(cls, tx) -> None:
        """
        category constraint query
        @param cls is the class
        @param tx is the transaction
        @returns none
        """
        query = ("CREATE CONSTRAINT ON (e:Category) ASSERT e.category IS UNIQUE")
        result = tx.run(query)


    @classmethod
    def create_protein_contraints(cls, tx) -> None:
        """
        protein constraint query
        @param cls is the class
        @param tx is the transaction
        @returns none
        """
        query = ("CREATE CONSTRAINT ON (p:Protein) ASSERT p.protein IS UNIQUE")
        result = tx.run(query)

    @classmethod
    def create_pathway_contraints(cls, tx) -> None:
        """
        protein constraint query
        @param cls is the class
        @param tx is the transaction
        @returns none
        """
        query = ("CREATE CONSTRAINT ON (a:Pathway) ASSERT a.ReactomeID IS UNIQUE")
        result = tx.run(query)

    @classmethod
    def create_root_contraints(cls, tx) -> None:
        """
        root constraint query
        @param cls is the class
        @param tx is the transaction
        @returns none
        """
        query = ("CREATE CONSTRAINT ON (a:CVD_Drugs) ASSERT a.name IS UNIQUE")
        result = tx.run(query)

    @classmethod
    def create_pmid_contraints(cls, tx) -> None:
        """
        pmid constraint query
        @param cls is the class
        @param tx is the transaction
        @returns none
        """
        query = ("CREATE CONSTRAINT ON (a:PMID) ASSERT a.id IS UNIQUE")
        result = tx.run(query)


    def run_constraints(self) -> None:
        """
        run all class constraints
        @param self
        @return none
        """
        tasks = [self.create_category_contraints, self.create_drug_categories, self.create_protein_contraints, self.create_pathway_contraints, self.create_root_contraints, self.create_pmid_contraints]
        for element in tqdm(tasks, desc = 'Creating Constraints: '):
            self.driver.session().write_transaction(element)

    ###############################################################################################
    #root node deployment

    @classmethod
    def create_cvd_node(cls, tx) -> None:
        '''
        query to create main node
        @param cls is the class
        @param tx is the transaction
        @returns none
        '''
        query = ("CREATE (d: CVD_Drugs {name : 'CVD Drugs'})")
        result = tx.run(query)

    def run_cvd_node(self) -> None:
        """
        create root cvd node
        @param self
        @return none
        """
        for i in tqdm(range(1), desc = "Creating CVD Node: "):
            self.driver.session().write_transaction(self.create_cvd_node)

    ###############################################################################################
    #protein pathway deployment

    @classmethod
    def create_proteins(cls, tx, protein, CM, ARR, CHD, VD, IHD, CCS, VOO, OHD) -> None:
        """
        creates proteins from caseolap csv
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("CREATE (:Protein {protein: $protein, CM: $CM, ARR: $ARR, CHD: $CHD, VD: $VD, IHD: $IHD, CCS: $CCS, VOO: $VOO, OHD: $OHD})")
        tx.run(query, protein = protein, CM = CM, ARR = ARR, CHD = CHD, VD = VD, IHD = IHD, CCS = CCS, VOO = VOO, OHD = OHD)

    @classmethod
    def create_pathways(cls, tx, pathway) -> None:
        """
        creates pathways from caseolap csv
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("MERGE (:Pathway {ReactomeID: $pathway})")
        tx.run(query, pathway = pathway)

    def run_proteins_pathways(self) -> None:
        """
        run the protein and pathway creation
        @param self
        @return none
        """
        for i in tqdm(range(len(self.caseolap_df['protein'])), desc = "Deploying Proteins: "):
            self.driver.session().write_transaction(self.create_proteins, self.caseolap_df.iloc[i]['protein'], self.caseolap_df.iloc[i]['CM'], self.caseolap_df.iloc[i]['ARR'], self.caseolap_df.iloc[i]['CHD'], self.caseolap_df.iloc[i]['VD'], self.caseolap_df.iloc[i]['IHD'], self.caseolap_df.iloc[i]['CCS'], self.caseolap_df.iloc[i]['VOO'], self.caseolap_df.iloc[i]['OHD'])

        for i in tqdm(range(len(self.caseolap_df['protein'])), desc = "Deploying Pathways: "):
            for j in range(len(self.caseolap_df['pathway'][i])):
                self.driver.session().write_transaction(self.create_pathways, self.caseolap_df['pathway'][i][j])
            
    ###############################################################################################
    #drug, category node deployment

    @classmethod
    def create_drug_categories(cls, tx) -> None:
        """
        drug and category node creation
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("LOAD CSV WITH HEADERS FROM 'file:///drug-category-target%20-%20drug-category-target.csv' AS row "
                 "MERGE (d:Drug {name: row.Name}) "
                 "MERGE (c:Category {category: row.Category})")
        tx.run(query)
    
    def run_drug_categories(self) -> None:
        """
        run drug and category deployment
        @param self
        @return none
        """
        for i in tqdm(range(1), desc = "Deploying Drugs and Categories: "):
            self.driver.session().write_transaction(self.create_drug_categories)

    ###############################################################################################
    #pmid node deployment

    @classmethod
    def create_pmid(cls, tx, pmid) -> None:
        '''
        query to run pmids
        @param cls is class
        @param tx is transaction used for in neo4j library
        @param pmid is the pmid number
        '''
        query = ('MERGE (p:PMID {id: $pmid})')
        tx.run(query, pmid = pmid)


    def run_pmid(self) -> None:
        '''
        deploys the pmids
        @param self
        @return none
        '''
        #iterate through the pmids
        for i in tqdm(range(len(self.pmid_df['pmid'])), desc = "Deploying PMIDs: "):
            self.driver.session().write_transaction(self.create_pmid, self.pmid_df['pmid'][i])

    ###############################################################################################
    #data preprocessing

    @classmethod
    def get_protein_list(cls, tx) -> any:
        """
        get list of proteins for each drug
        @param self
        @return a data table
        """
        query = ("LOAD CSV WITH HEADERS FROM 'file:///drug-category-target%20-%20drug-category-target.csv' AS row "
                 "RETURN row.Name as name, row.Category AS category, row.`Target Protein (UniprotID)` AS UniProtID")
        result = tx.run(query)
        return result.data()

    def run_protein_list(self) -> None:
        """
        creates a csv from the data
        @param self
        @return None
        """
        #grab the table from the query
        result = self.driver.session().write_transaction(self.get_protein_list)
        #create dataframe
        result = pd.DataFrame(result)

        #create the string to a list
        for element in tqdm(range(len(result['UniProtID'])), desc = "Parsing CSV"):
            result['UniProtID'][element] = str(result['UniProtID'][element]).split(',')
            for i in range(len(result['UniProtID'][element])):
                result['UniProtID'][element][i] = str(result['UniProtID'][element][i]).strip()
        self.df = result

    ###############################################################################################
    #relationships deployment

    @classmethod
    def create_root_category_relationships(cls, tx) -> None:
        """
        query for relationships of the drugs and categories
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("MATCH (d:CVD_Drugs) WHERE d.name = 'CVD Drugs' " 
                 "MATCH (c:Category) "
                 "MERGE (d)-[r:CATEGORY]->(c)")
        tx.run(query)

    @classmethod
    def create_drug_category_relationships(cls, tx, drug, category) -> None:
        """
        query for relationships of the drugs and categories
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("MATCH (d:Drug) WHERE d.name = $drug " 
                 "MATCH (c:Category) WHERE c.category = $category "
                 "MERGE (d)-[r:CANDIDATE_OF]->(c)")
        tx.run(query, drug = drug, category = category)


    @classmethod
    def create_drug_protein_relationships(cls, tx, drug, protein) -> None:
        """
        query for relationships of categories and proteins
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("MATCH (c:Drug) WHERE c.name = $drug "
                 "MATCH (p:Protein) WHERE p.protein = $protein "
                 "MERGE (c)-[r:TARGET]->(p)")
        tx.run(query, drug = drug, protein = protein)

    @classmethod
    def create_protein_pathway_relationships(cls, tx, protein, pathway) -> None:
        """
        query for relationships of categories and proteins
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("MATCH (p:Protein) WHERE p.protein = $protein "
                 "MATCH (a:Pathway) WHERE a.ReactomeID = $pathway "
                 "MERGE (p)-[r:ASSOCIATED_WITH]-(a)")
        tx.run(query, protein = protein, pathway = pathway)

    def run_relationships(self) -> None:
        """
        runs all relationships
        @param self is the object of the class
        """
        #process uniprot ids in drug, category, targets
        self.run_protein_list()
        
        #create root relationship 
        self.driver.session().write_transaction(self.create_root_category_relationships)

        for i in tqdm(range(len(self.df['category'])), desc = "Deploying Drug-Category/Drug-Protein Relationships: "):
            #create drug category relationship
            self.driver.session().write_transaction(self.create_drug_category_relationships, self.df['name'][i], self.df['category'][i])
            for j in range(len(self.df['UniProtID'][i])):
                #create drug protein relationship
                self.driver.session().write_transaction(self.create_drug_protein_relationships, self.df['name'][i], self.df['UniProtID'][i][j])

        for i in tqdm(range(len(self.caseolap_df['protein'])), desc = "Deploying Protein-Pathway Relationships: "):
            for j in range(len(self.caseolap_df['pathway'][i])):
                #create protein pathway relationship
                self.driver.session().write_transaction(self.create_protein_pathway_relationships, self.caseolap_df['protein'][i], self.caseolap_df['pathway'][i][j])


    ###############################################################################################
    #pmid-proteins relationships deployment

    @classmethod
    def create_protein_pmid_relationships(cls, tx, protein, pmid) -> None:
        """
        query for relationships of pmid and proteins
        @param cls is the class
        @param tx is the transaction
        @return none
        """
        query = ("MATCH (p:Protein) WHERE p.protein = $protein "
                 "MATCH (a:PMID) WHERE a.id = $pmid "
                 "MERGE (p)-[r:PMID_TARGET]->(a)")
        tx.run(query, protein = protein, pmid = pmid)

    def run_pmid_protein_relationships(self) -> None:
        '''
        runs protein pmid relationships
        @param self 
        @return none
        '''
        
        for i in tqdm(range(len(self.pmid_df['pmid'])), desc = "Deploying PMID-Protein Relationships: "):
            for j in range(len(self.pmid_df['unid'][i])):
                self.driver.session().write_transaction(self.create_protein_pmid_relationships, self.pmid_df['unid'][i][j], self.pmid_df['pmid'][i])



In [3]:
graph = graph_creation()


Searching Pathways in Caseolap CSV (__init__): 100%|██████████| 103/103 [00:47<00:00,  2.15it/s]


In [4]:
graph.run_constraints()

Creating Constraints: 100%|██████████| 6/6 [00:00<00:00, 16.78it/s]


In [5]:
graph.run_cvd_node()

Creating CVD Node: 100%|██████████| 1/1 [00:00<00:00, 49.49it/s]


In [6]:
graph.run_proteins_pathways()

Deploying Pathways: 100%|██████████| 103/103 [00:02<00:00, 40.02it/s]


In [7]:
graph.run_drug_categories()

Deploying Drugs and Categories: 100%|██████████| 1/1 [00:00<00:00, 21.13it/s]


In [8]:
graph.run_pmid()

Deploying PMIDs: 100%|██████████| 18819/18819 [01:43<00:00, 182.56it/s]


In [9]:
graph.run_pmid_protein_relationships()
graph.run_relationships()

Parsing CSV: 100%|██████████| 161/161 [00:00<00:00, 8482.07it/s]
Deploying Drug-Category/Drug-Protein Relationships: 100%|██████████| 161/161 [00:02<00:00, 61.16it/s]
Deploying Protein-Pathway Relationships: 100%|██████████| 103/103 [00:03<00:00, 30.65it/s]
